In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import torch
from datasets import load_dataset
from datasets import get_dataset_split_names
xsum_dataset = load_dataset("xsum")

In [ ]:
# from transformers import 
from transformers import PegasusForConditionalGeneration,PegasusTokenizer
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

base_model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
from dataset import *
num_samples=204045
val_dataset = Dataset("val", tokenizer,xsum_dataset["train"]["document"][0:] , xsum_dataset["train"]["summary"][0:])

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 5, shuffle = False)
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
model = base_model.to(device)

from tqdm import tqdm
import gc
import pandas as pd
import pickle



def beam_search_step(batch, tokenizer, base_model, device):
    # 1 - beam search
    if True:
        summary_ids = base_model.generate(
            batch["text_inputs"]['input_ids'],
            attention_mask = batch["text_inputs"]["attention_mask"],
            num_beams = 5,
            num_return_sequences = 5,
            max_length = 64
        )

    generated = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    del summary_ids
    gc.collect()

    return generated


def get_summaries(tokenizer, val_loader, base_model, device):
    val_texts = []
    val_summaries = []
    val_labels = []

    for idx, batch in tqdm(enumerate(val_loader)):
        for k in batch["text_inputs"].keys():
            batch["text_inputs"][k] = batch["text_inputs"][k].to(device)
            if len(batch["text_inputs"][k].shape) > 2:
                batch["text_inputs"][k] = batch["text_inputs"][k].squeeze(1)

        model.zero_grad()
        val_texts += batch["text"]

        raw_summaries = beam_search_step(batch, tokenizer, base_model, device)
        
        summaries = []
        for i in range(len(batch["text"])):
            summaries.append("|".join(raw_summaries[i*5:(i+1)*5]))

        val_summaries += summaries

        labels = batch["summary"]
        val_labels += labels
        df = pd.DataFrame({"text": batch["text"],"summaries":summaries,"labels":labels})
        df.to_csv("candidates_1.csv",mode='a',index=False,header=False)

        # with open(f"candidates_txt", 'a+b') as fp:
        #     pickle.dump(val_texts,fp)
        # with open(f"candidates_summary", 'a+b') as fp:
        #     pickle.dump(val_summaries,fp)
        # with open(f"candidates_label", 'a+b') as fp:
        #     pickle.dump(val_labels,fp)

    # print(len(val_texts), len(val_summaries), len(val_summaries[0]), len(val_labels))

    # return val_texts, val_summaries, val_labels

In [ ]:

get_summaries(tokenizer, val_loader, model, device)

In [ ]:
# import pandas as pd
# df = pd.read_csv("candidates_1.csv",header=None)
# df.head()